In [220]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import numpy as np

In [221]:
import os
import cv2

In [222]:
import pandas as pd

In [223]:
sources = ["https://towardsdatascience.com/build-your-own-convolution-neural-network-in-5-mins-4217c2cf964f"]

In [224]:
image_root = 'Making-Learning/dataset/jaffe/'
image_title = os.listdir(image_root)
image_dirs = [image_root + x for x in os.listdir(image_root)]

In [225]:
H = 128
W = 128
def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    dimension = (H,W)
    resized = cv2.resize(img, dimension)
    return resized
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, 1, H, W), dtype=np.uint8)
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image
        if i%50 == 0: print('Processed {} of {}'.format(i, count))
    return data

In [226]:
image_dirs.remove('Making-Learning/dataset/jaffe/.ipynb_checkpoints')
image_dirs.remove('Making-Learning/dataset/jaffe/Untitled.ipynb')

In [227]:
images = prep_data(image_dirs)

Processed 0 of 213
Processed 50 of 213
Processed 100 of 213
Processed 150 of 213
Processed 200 of 213


In [228]:
image_labels = np.array([title[3:5] for title in image_title])

In [229]:
labels = []
for i, label in enumerate(image_labels):
    if label == 'AN':
        labels.append(0)
    if label == 'DI':
        labels.append(1)
    if label == 'FE':
        labels.append(2)
    if label == 'SU':
        labels.append(3)
    if label == 'SA':
        labels.append(4)
    if label == 'HA':
        labels.append(5)
    if label == 'NE':
        labels.append(6)

In [230]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
 
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.20, random_state=69)

X_train = X_train.reshape(X_train.shape[0], H, W, 1)
X_test = X_test.reshape(X_test.shape[0], H, W, 1)

# Model 1

In [231]:
def CNN1(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(H,W,1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adadelta',metrics=['accuracy'])
    return model

In [232]:
cnn_model = CNN1(7)

In [233]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_98 (Conv2D)           (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_78 (Dropout)         (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 246016)            0         
_________________________________________________________________
dense_90 (Dense)             (None, 128)               31490176  
_________________________________________________________________
dropout_79 (Dropout)         (None, 128)               0         
__________

In [234]:
cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30,batch_size=8, verbose=2)

Train on 170 samples, validate on 43 samples
Epoch 1/30
 - 20s - loss: 14.1270 - acc: 0.1235 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 2/30
 - 18s - loss: 13.5592 - acc: 0.1588 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 3/30
 - 19s - loss: 14.2218 - acc: 0.1176 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 4/30
 - 19s - loss: 13.8426 - acc: 0.1412 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 5/30
 - 16s - loss: 13.8426 - acc: 0.1412 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 6/30
 - 16s - loss: 14.0322 - acc: 0.1294 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 7/30
 - 17s - loss: 13.8426 - acc: 0.1412 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 8/30
 - 17s - loss: 13.9374 - acc: 0.1353 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 9/30
 - 18s - loss: 13.8229 - acc: 0.1412 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 10/30
 - 17s - loss: 13.4634 - acc: 0.1647 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 11/30
 - 17s - loss: 13.8426 - acc: 0.1412 - val_loss: 13.4942 - val_acc: 0.1628
Epoch 1

In [235]:
scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 90.70%


## Model 2

In [210]:
def CNN2(num_classes):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=(H,W,1)))
    model.add(BatchNormalization())
    model.add(AveragePooling2D())
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adadelta',metrics=['accuracy'])
    return model

In [211]:
cnn_model2 = CNN2(7)

In [212]:
cnn_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_93 (Conv2D)           (None, 124, 124, 64)      1664      
_________________________________________________________________
batch_normalization_v1_23 (B (None, 124, 124, 64)      256       
_________________________________________________________________
average_pooling2d_14 (Averag (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 58, 58, 32)        51232     
_________________________________________________________________
batch_normalization_v1_24 (B (None, 58, 58, 32)        128       
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 27, 27, 32)        9248      
__________

In [213]:
cnn_model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30,batch_size=8, verbose=2)

Train on 170 samples, validate on 43 samples
Epoch 1/30
 - 24s - loss: 7.2038 - acc: 0.1176 - val_loss: 10.0676 - val_acc: 0.1628
Epoch 2/30
 - 16s - loss: 4.0125 - acc: 0.1529 - val_loss: 3.3153 - val_acc: 0.2326
Epoch 3/30
 - 16s - loss: 2.3313 - acc: 0.1588 - val_loss: 2.6221 - val_acc: 0.1860
Epoch 4/30
 - 16s - loss: 1.9954 - acc: 0.2176 - val_loss: 2.5093 - val_acc: 0.1628
Epoch 5/30
 - 16s - loss: 1.9723 - acc: 0.1824 - val_loss: 2.2752 - val_acc: 0.0930
Epoch 6/30
 - 16s - loss: 1.9185 - acc: 0.2235 - val_loss: 1.9058 - val_acc: 0.1860
Epoch 7/30
 - 16s - loss: 1.9797 - acc: 0.2059 - val_loss: 2.0506 - val_acc: 0.2791
Epoch 8/30
 - 16s - loss: 1.9598 - acc: 0.1941 - val_loss: 1.9008 - val_acc: 0.2326
Epoch 9/30
 - 16s - loss: 1.9251 - acc: 0.1588 - val_loss: 1.8980 - val_acc: 0.2326
Epoch 10/30
 - 16s - loss: 1.9876 - acc: 0.1882 - val_loss: 1.8721 - val_acc: 0.1395
Epoch 11/30
 - 15s - loss: 1.9764 - acc: 0.2294 - val_loss: 1.8439 - val_acc: 0.2093
Epoch 12/30
 - 16s - loss: 1

In [214]:
scores = cnn_model2.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 81.40%


# Model 3

In [200]:
def CNN3(num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(W, H, 1)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*2*32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*2*32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(2*2*2*32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2*2*2*32, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(2*2*2*32, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2*2*32, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2*32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])
    return model

In [201]:
cnn_model3 = CNN3(7)
cnn_model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_82 (Conv2D)           (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 126, 126, 32)      9248      
_________________________________________________________________
batch_normalization_v1_14 (B (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_65 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 63, 63, 64)        18496     
_________________________________________________________________
batch_normalization_v1_15 (B (None, 63, 63, 64)        256       
__________

In [202]:
cnn_model3.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30,batch_size=8, verbose=2)

Train on 170 samples, validate on 43 samples
Epoch 1/30
 - 26s - loss: 11.2035 - acc: 0.1235 - val_loss: 5.4284 - val_acc: 0.2326
Epoch 2/30
 - 22s - loss: 10.7625 - acc: 0.1882 - val_loss: 9.0751 - val_acc: 0.1860
Epoch 3/30
 - 20s - loss: 10.6141 - acc: 0.1294 - val_loss: 13.0640 - val_acc: 0.0930
Epoch 4/30
 - 20s - loss: 11.1744 - acc: 0.1529 - val_loss: 13.9091 - val_acc: 0.0930
Epoch 5/30
 - 20s - loss: 10.4259 - acc: 0.1471 - val_loss: 6.0177 - val_acc: 0.1395
Epoch 6/30
 - 20s - loss: 8.6015 - acc: 0.1941 - val_loss: 5.0060 - val_acc: 0.0698
Epoch 7/30
 - 22s - loss: 7.4830 - acc: 0.1294 - val_loss: 2.7470 - val_acc: 0.0698
Epoch 8/30
 - 21s - loss: 6.0156 - acc: 0.1647 - val_loss: 2.9747 - val_acc: 0.0698
Epoch 9/30
 - 22s - loss: 6.8787 - acc: 0.1471 - val_loss: 4.8016 - val_acc: 0.1395
Epoch 10/30
 - 22s - loss: 5.9504 - acc: 0.1882 - val_loss: 3.0044 - val_acc: 0.1163
Epoch 11/30
 - 22s - loss: 4.7840 - acc: 0.1176 - val_loss: 2.3030 - val_acc: 0.1628
Epoch 12/30
 - 22s - l

In [203]:
scores = cnn_model3.evaluate(X_test, y_test, verbose=0)
print("Classification Error: %.2f%%" % (100-scores[1]*100))

Classification Error: 88.37%
